In [ ]:
from pymatting import *
from PIL import Image
import json
import pandas as pd

import os
from joblib import Parallel, delayed
from tqdm import tqdm
from tqdm_joblib import tqdm_joblib

# Using PyMatting

In [23]:


# Suponha que as funções abaixo já estejam definidas no seu código:
# load_image, estimate_alpha_cf, estimate_foreground_ml, blend, save_image, make_grid
# E também a variável global "scale" (se aplicável)

def process_single_image(row, trimaps_dir, textures_dir, output_dir, grid_dir):
    """
    Processa uma única imagem:
      - Carrega o mask, as texturas (foreground e background)
      - Executa os passos de matting
      - Salva o resultado (e o grid, se grid_dir for fornecido)
    """
    # Extração dos nomes de arquivos
    file_name = row["file_name"]
    foreground_texture_name = row["foreground_texture_name"]
    background_texture_name = row["background_texture_name"]
    print(f"Processando {file_name}")

    # Monta os caminhos dos arquivos
    mask_path = os.path.join(trimaps_dir, file_name)
    foreground_texture_path = os.path.join(textures_dir, foreground_texture_name)
    background_texture_path = os.path.join(textures_dir, background_texture_name)

    # Carrega as imagens
    mask = load_image(mask_path, "GRAY", size=1.0, resample="nearest")
    foreground_texture = load_image(foreground_texture_path, "RGB", size=1.0, resample="box")
    background_texture = load_image(background_texture_path, "RGB", size=1.0, resample="box")

    # Processamento: estima alpha, foreground estimado e realiza o blend
    alpha = estimate_alpha_knn(foreground_texture, mask)
    estimated_foreground = estimate_foreground_ml(foreground_texture, alpha)
    matting = blend(estimated_foreground, background_texture, alpha)

    # Salva a imagem processada
    save_image(os.path.join(output_dir, file_name), matting)

    # Se grid_dir for fornecido, cria e salva o grid com as imagens
    if grid_dir:
        images_grid = [foreground_texture, background_texture, alpha, matting]
        grid = make_grid(images_grid)
        save_image(os.path.join(grid_dir, file_name), grid)


def vessel_shape_with_matte(images_metadata, trimaps_dir, textures_dir, output_dir, grid_dir=None):
    """
    Função principal que prepara os diretórios de saída e paraleliza o processamento
    utilizando a joblib.
    """
    # Cria os diretórios de saída, se não existirem
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    if grid_dir and not os.path.exists(grid_dir):
        os.makedirs(grid_dir)

    # Usando joblib para processar em paralelo com barra de progresso integrada pelo tqdm_joblib
    with tqdm_joblib(tqdm(desc="Processando", total=images_metadata.shape[0])):
        Parallel(n_jobs=-1)(
            delayed(process_single_image)(row, trimaps_dir, textures_dir, output_dir, grid_dir)
            for idx, row in images_metadata.iterrows()
        )




In [24]:
images_metadata = pd.read_json("curves/metadata/images_metadata.json")
images_metadata['foreground_texture_name'] = images_metadata['foreground_texture_name'].apply(lambda x: x + ".JPEG")
images_metadata['background_texture_name'] = images_metadata['background_texture_name'].apply(lambda x: x + ".JPEG")



In [26]:
trimaps_dir = "curves/trimaps"
textures_dir = "/home/wesleygalvao/Insync/wesleygalv@gmail.com/Google Drive - Shared with me/Mestrado - Visão Computacional/Pesquisa/Experimentos/Datasets/ImageNet/ILSVRC2012_img_val_cropped"
output_dir = "curves/images_matting"
grid_dir = "curves/grids_matting"

# vessel_shape_with_matte(images_metadata, trimaps_dir, textures_dir, output_dir, grid_dir)
vessel_shape_with_matte(images_metadata, trimaps_dir, textures_dir, output_dir, grid_dir)





Processando:   0%|          | 0/70000 [00:00<?, ?it/s]




  0%|          | 0/70000 [00:00<?, ?it/s]/home/wesleygalvao/anaconda3/envs/mestrado_rapids/lib/python3.12/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)
/home/wesleygalvao/anaconda3/envs/mestrado_rapids/lib/python3.12/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)
/home/wesleygalvao/anaconda3/envs/mestrado_rapids/lib/python3.12/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found

KeyboardInterrupt: 

# Using Gaussian Blur Filter

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from skimage.draw import disk
from scipy.ndimage import distance_transform_edt
import scipy.ndimage as ndi

In [4]:
def generate_trimap(mask, border_thickness=4):
    """Generate trimap image from mask image"""

    # mask = (mask > 127).astype(np.uint8)

    foreground = distance_transform_edt(mask) > border_thickness//2
    background = distance_transform_edt(1-mask) > border_thickness//2
    img_border = np.logical_and(1-foreground, 1-background)

    tri_map = np.zeros(mask.shape)
    tri_map[img_border] = 1
    tri_map[foreground>0] = 2

    return tri_map

def mix_images(foreground, background, mask):
    """Mix two images using a mask and
    Parameters
    ----------
    foreground: numpy array
        Image to be overlayed.
    background: numpy array
        Image to be overlayed on.
    mask: numpy array
        Binary mask to be used as overlay.

    Returns
    -------
    img_mix: numpy array
    """

    mask = np.expand_dims(mask, 2)
    img_mix = foreground*mask + background*(1-mask)
    img_mix = img_mix.astype(np.uint8)

    return img_mix

def blend(foreground, background, mask, sigma=1):
    """Blend two images using a mask and Gaussian blur.

    Parameters
    ----------
    foreground: numpy array
        Image to be overlayed.
    background: numpy array
        Image to be overlayed on.
    mask: numpy array
        Binary mask to be used as overlay.
    sigma: float
        Standard deviation of the Gaussian filter.

    Returns
    -------
    img_blend: numpy array
    """

    alpha_fore = ndi.gaussian_filter(mask.astype(float) , sigma=sigma)

    alpha_fore = alpha_fore/alpha_fore.max()
    alpha_fore = np.expand_dims(alpha_fore, 2)

    img_blend = foreground*alpha_fore + background*(1-alpha_fore)
    img_blend = img_blend.astype(np.uint8)

    return img_blend

def vessel_shape_with_matte(images_metadata, mask_dir, textures_dir, output_dir, grid_dir=None):
    """
    Generate VesselShape dataset with matted images using Gaussian blur filter. The matting is performed to avoid
    hard transitions between the foreground and background textures.

    Parameters
    ----------
    images_metadata: pandas DataFrame
        DataFrame containing the metadata of the images to be processed.
    mask_dir: str
        Directory containing the mask images.
    textures_dir: str
        Directory containing the texture images.
    output_dir: str
        Directory where the output images will be saved.
    grid_dir: str, optional
        Directory where the grid images will be saved. Default is None.

    Returns
    -------
    None
    """

    # Create output directories if they do not exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    if grid_dir and not os.path.exists(grid_dir):
        os.makedirs(grid_dir)

    for idx, row in tqdm(images_metadata.iterrows(), desc="Processing", total=images_metadata.shape[0]):

        # Load the mask, foreground and background textures
        mask = np.array(Image.open(os.path.join(mask_dir, row['file_name'])).convert('L'))
        mask = (mask > 127).astype(np.uint8)
        foreground_texture = np.array(Image.open(os.path.join(textures_dir, row['foreground_texture_name'])))
        background_texture = np.array(Image.open(os.path.join(textures_dir, row['background_texture_name'])))

        random_sigma = np.random.uniform(1, 2) # Random sigma value for the Gaussian filter
        img_blend = blend(foreground_texture, background_texture, mask, sigma=random_sigma)

        # Save the blended image
        Image.fromarray(img_blend).save(os.path.join(output_dir, row['file_name']))

        # Save the grid image if grid_dir is provided
        if grid_dir:
            grid = np.hstack((foreground_texture, background_texture, img_blend))
            Image.fromarray(grid).save(os.path.join(grid_dir, row['file_name']))


In [4]:
images_metadata = pd.read_json("curves/metadata/images_metadata.json")
images_metadata['foreground_texture_name'] = images_metadata['foreground_texture_name'].apply(lambda x: x + ".JPEG")
images_metadata['background_texture_name'] = images_metadata['background_texture_name'].apply(lambda x: x + ".JPEG")

In [26]:
masks_dir = "curves/labels"
textures_dir = "/home/wesleygalvao/Insync/wesleygalv@gmail.com/Google Drive - Shared with me/Mestrado - Visão Computacional/Pesquisa/Experimentos/Datasets/ImageNet/ILSVRC2012_img_val_cropped"
output_dir = "curves/images_matting"
grid_dir = "curves/grids_matting"

vessel_shape_with_matte(images_metadata, masks_dir, textures_dir, output_dir, grid_dir)

Processing:   0%|          | 242/70000 [00:10<51:46, 22.45it/s]


KeyboardInterrupt: 

# Using Gaussian Blur Filter with GPU


In [2]:
import os
import random
import numpy as np
from PIL import Image
from concurrent.futures import ProcessPoolExecutor, as_completed
import tqdm
from pymatting import *
import pandas as pd

# Importando CuPy e cupyx para operações na GPU
import cupy as cp
import cupyx.scipy.ndimage as cndi

# set seed for reproducibility
random.seed(0)

# --------------------------
# Funções de processamento GPU
# --------------------------

def generate_trimap(mask, border_thickness=4):
    """
    Gera o trimap a partir de uma máscara usando operações aceleradas na GPU.
    """
    mask_gpu = cp.asarray(mask)  # Converte para cupy array
    # Calcula a distância (usando a versão GPU do distance_transform_edt)
    foreground = cndi.distance_transform_edt(mask_gpu) > (border_thickness // 2)
    background = cndi.distance_transform_edt(1 - mask_gpu) > (border_thickness // 2)
    img_border = cp.logical_and(1 - foreground, 1 - background)

    tri_map = cp.zeros(mask_gpu.shape, dtype=cp.uint8)
    tri_map[img_border] = 1
    tri_map[foreground] = 2

    return cp.asnumpy(tri_map)  # Retorna de volta para CPU como numpy array

def mix_images(foreground, background, mask):
    """
    Mistura duas imagens usando uma máscara (operações na GPU).
    """
    fg_gpu = cp.asarray(foreground)
    bg_gpu = cp.asarray(background)
    mask_gpu = cp.asarray(mask)
    mask_gpu = cp.expand_dims(mask_gpu, axis=2)
    img_mix = fg_gpu * mask_gpu + bg_gpu * (1 - mask_gpu)
    return cp.asnumpy(img_mix.astype(cp.uint8))

def blend(foreground, background, mask, sigma=1):
    """
    Realiza o blend (mesclagem) de duas imagens usando um filtro Gaussiano,
    com os cálculos realizados na GPU.
    """
    # Converte as imagens para cupy arrays
    fg_gpu = cp.asarray(foreground)
    bg_gpu = cp.asarray(background)
    mask_gpu = cp.asarray(mask)

    # Aplica o filtro gaussiano usando a versão GPU
    alpha_fore = cndi.gaussian_filter(mask_gpu.astype(cp.float32), sigma=sigma)
    max_val = cp.max(alpha_fore)
    if max_val > 0:
        alpha_fore = alpha_fore / max_val
    else:
        alpha_fore = cp.zeros_like(alpha_fore)

    alpha_fore = cp.expand_dims(alpha_fore, axis=2)

    img_blend = fg_gpu * alpha_fore + bg_gpu * (1 - alpha_fore)
    return cp.asnumpy(img_blend.astype(cp.uint8)), cp.asnumpy(alpha_fore)

# --------------------------
# Função de processamento de cada imagem (I/O + GPU)
# --------------------------

def process_row(row, mask_dir, textures_dir, output_dir, grid_dir):
    """
    Lê os arquivos de máscara e texturas, aplica o blend usando a GPU e salva os resultados.
    Essa função será executada em paralelo.

    Parâmetros de `row` (um dicionário):
      - 'file_name': nome do arquivo de máscara (e nome base para salvar a imagem final)
      - 'foreground_texture_name': nome do arquivo da textura do primeiro plano
      - 'background_texture_name': nome do arquivo da textura do fundo
    """
    try:
        # Caminhos completos para os arquivos
        mask_path = os.path.join(mask_dir, row['file_name'])
        fg_path = os.path.join(textures_dir, row['foreground_texture_name'])
        bg_path = os.path.join(textures_dir, row['background_texture_name'])

        # Leitura das imagens com PIL
        mask = np.array(Image.open(mask_path).convert('L'))
        # Cria uma máscara binária
        mask = (mask > 127).astype(np.uint8)

        foreground_texture = np.array(Image.open(fg_path))
        background_texture = np.array(Image.open(bg_path))

        # Escolhe sigma aleatório para o blur
        sigma = random.uniform(1, 2)
        # Processa o blend usando a GPU (função blend)
        img_blend, alpha_fore = blend(foreground_texture, background_texture, mask, sigma=sigma)

        # Salva a imagem blended
        output_path = os.path.join(output_dir, row['file_name'])
        Image.fromarray(img_blend).save(output_path)

        if grid_dir:
            alpha_fore = (alpha_fore * 255).astype(np.uint8)
            if alpha_fore.ndim == 3 and alpha_fore.shape[2] == 1:
                alpha_fore = np.repeat(alpha_fore, 3, axis=2)
            grid = np.hstack((foreground_texture, background_texture, alpha_fore, img_blend))
            grid_path = os.path.join(grid_dir, row['file_name'])
            Image.fromarray(grid).save(grid_path)

    except Exception as e:
        print(f"Erro ao processar {row.get('file_name', 'unknown')}: {e}")

# --------------------------
# Função principal que paraleliza o processamento
# --------------------------

def vessel_shape_with_matte(images_metadata, mask_dir, textures_dir, output_dir, grid_dir=None):
    """
    Processa o dataset, lendo as imagens e aplicando o blend com aceleração GPU.
    A leitura e escrita dos arquivos é paralelizada com multiprocessing.

    Parâmetros:
      - images_metadata: DataFrame do pandas contendo as colunas 'file_name',
        'foreground_texture_name' e 'background_texture_name'
      - mask_dir: diretório contendo as máscaras
      - textures_dir: diretório contendo as texturas
      - output_dir: diretório onde as imagens blended serão salvas
      - grid_dir: (opcional) diretório onde serão salvas as imagens em grid
    """
    # Cria os diretórios de saída, se não existirem
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    if grid_dir and not os.path.exists(grid_dir):
        os.makedirs(grid_dir)

    # Converte o DataFrame para uma lista de dicionários para facilitar o envio para os processos
    rows = images_metadata.to_dict('records')

    # Paraleliza o processamento (leitura + blend + salvamento) com ProcessPoolExecutor
    with ProcessPoolExecutor() as executor:
        futures = [
            executor.submit(process_row, row, mask_dir, textures_dir, output_dir, grid_dir)
            for row in rows
        ]
        # Barra de progresso única usando tqdm
        for _ in tqdm.tqdm(as_completed(futures), total=len(futures), desc="Processando"):
            pass


In [4]:
images_metadata = pd.read_json("curves/metadata/images_metadata.json")
images_metadata['foreground_texture_name'] = images_metadata['foreground_texture_name'].apply(lambda x: x + ".JPEG")
images_metadata['background_texture_name'] = images_metadata['background_texture_name'].apply(lambda x: x + ".JPEG")

In [5]:
masks_dir = "curves/labels"
textures_dir = "/home/wesleygalvao/Insync/wesleygalv@gmail.com/Google Drive - Shared with me/Mestrado - Visão Computacional/Pesquisa/Experimentos/Datasets/ImageNet/ILSVRC2012_img_val_cropped"
output_dir = "curves/images_matting"
grid_dir = "curves/grids_matting"

vessel_shape_with_matte(images_metadata, masks_dir, textures_dir, output_dir, grid_dir)

Processando: 100%|██████████| 70000/70000 [07:14<00:00, 161.07it/s]
